In [46]:
import cvxpy as cp
import numpy as np
import networkx as nx


inputnodes = [0,6]
inputedges = [(0,5),(0,3),(3,4),(4,5),(4,6),(5,2),(3,1),(1,6),(6,2)]

non_member = [3,4,5,6]
source = 0
recievers = [1,2]


preG = nx.Graph()
preG.add_nodes_from(inputnodes)
preG.add_edges_from(inputedges)
directE = []
for n, nbrs in preG.adj.items():
    for nbr, eattr in nbrs.items():
        directE.append((n,nbr))


G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(directE)

undirected_edges = set(inputedges)

vertices = np.arange(inputnodes[1]+1)
print(vertices)

edges = set(directE)
print(edges)


f_star = cp.Variable()  # Target flow rate


orientation_constraints = []
flow_constraints = []
equalrate_constraints = []

c =  {a : cp.Variable() for a in edges}

for a in edges:
    orientation_constraints += [c[a] >= 0]      #for every in 2E

for a1 in undirected_edges:         # for every in E
    a2 = (a1[1], a1[0]) 
    orientation_constraints += [ c[a1] + c[a2] == 1]


fia = {(i, a): cp.Variable() for a in edges for i in recievers}

for i in recievers: 
     for a in edges:
          flow_constraints += [fia[i,a] <= c[a]]
          flow_constraints += [fia[i,a] >= 0]


for i in recievers:
    for j in non_member:
        incoming_edges = [(u, v) for u, v in edges if v == int(j)] 
        outcoming_edges = [(u, v) for u, v in edges if u == int(j)] 
        incoming_flow = sum(fia[i, x] for x in incoming_edges)
        outcoming_flow = sum(fia[i, y] for y in outcoming_edges)
        flow_constraints += [incoming_flow == outcoming_flow]
        print(incoming_edges)

    incoming_flow_source = sum(fia[i, x] for x in edges if x[1] == source)  
    
    outcoming_flow_reciever = sum(fia[i, x] for x in edges if x[0] == int(i)) 
    incoming_flow_reciever = sum(fia[i, x] for x in edges if x[1] == int(i))  

    flow_constraints += [incoming_flow_source == 0]
    flow_constraints += [outcoming_flow_reciever == 0]
    equalrate_constraints += [f_star == incoming_flow_reciever]


objective = cp.Maximize(f_star)    
constraints = orientation_constraints + flow_constraints + equalrate_constraints
problem = cp.Problem(objective, constraints)


problem.solve()

print("Optimal Target Flow Rate (f*):", f_star.value)



[0 1 2 3 4 5 6]
{(6, 2), (3, 4), (4, 3), (3, 1), (6, 1), (5, 4), (0, 3), (4, 6), (6, 4), (3, 0), (4, 5), (5, 0), (2, 6), (0, 5), (1, 6), (2, 5), (1, 3), (5, 2)}
[(4, 3), (0, 3), (1, 3)]
[(3, 4), (5, 4), (6, 4)]
[(4, 5), (0, 5), (2, 5)]
[(4, 6), (2, 6), (1, 6)]
[(4, 3), (0, 3), (1, 3)]
[(3, 4), (5, 4), (6, 4)]
[(4, 5), (0, 5), (2, 5)]
[(4, 6), (2, 6), (1, 6)]
Optimal Target Flow Rate (f*): 1.9999999999932954


In [7]:
import cvxpy as cp
import numpy as np
import networkx as nx



inputnodes = [0,6]
inputedges = [(0,5),(0,3),(3,4),(4,5),(4,6),(5,2),(3,1),(1,6),(6,2)]

C = {
    (0,5):1,
    (0,3):1,
    (3,4):1,
    (4,5):1,
    (4,6):1,
    (5,2):1,
    (3,1):1,
    (1,6):1,
    (6,2):1
}

non_member = [3,4,5,6]
source = 0
recievers = [1,2]


preG = nx.Graph()
preG.add_nodes_from(inputnodes)
preG.add_edges_from(inputedges)
directE = []
for n, nbrs in preG.adj.items():
    for nbr, eattr in nbrs.items():
        directE.append((n,nbr))


G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(directE)

undirected_edges = set(inputedges)

vertices = np.arange(inputnodes[1]+1)
print(vertices)

edges = set(directE)
print(edges)


f_star = cp.Variable()  # Target flow rate


orientation_constraints = []
flow_constraints = []
equalrate_constraints = []

c =  {a : cp.Variable() for a in edges}

for a in edges:
    orientation_constraints += [c[a] >= 0]      #for every in 2E

for a1 in undirected_edges:         # for every in E
    a2 = (a1[1], a1[0]) 
    orientation_constraints += [ c[a1] + c[a2] == C[a1]]


fia = {(i, a): cp.Variable() for a in edges for i in recievers}

for i in recievers: 
     for a in edges:
          flow_constraints += [fia[i,a] <= c[a]]
          flow_constraints += [fia[i,a] >= 0]


for i in recievers:
    for j in non_member:
        incoming_edges = [(u, v) for u, v in edges if v == int(j)] 
        outcoming_edges = [(u, v) for u, v in edges if u == int(j)] 
        incoming_flow = sum(fia[i, x] for x in incoming_edges)
        outcoming_flow = sum(fia[i, y] for y in outcoming_edges)
        flow_constraints += [incoming_flow == outcoming_flow]
        print(incoming_edges)

    incoming_flow_source = sum(fia[i, x] for x in edges if x[1] == source)  
    
    outcoming_flow_reciever = sum(fia[i, x] for x in edges if x[0] == int(i)) 
    incoming_flow_reciever = sum(fia[i, x] for x in edges if x[1] == int(i))  

    flow_constraints += [incoming_flow_source == 0]
    flow_constraints += [outcoming_flow_reciever == 0]
    equalrate_constraints += [f_star == incoming_flow_reciever]


objective = cp.Maximize(f_star)    
constraints = orientation_constraints + flow_constraints + equalrate_constraints
problem = cp.Problem(objective, constraints)


problem.solve()

print("Optimal Target Flow Rate (f*):", f_star.value)

for i in recievers:
    for edge in edges:
        ivalue = i
        evalue = edge
        flow = fia[i,edge].value
        print("session: ", ivalue, "edge: ", evalue, ", flow: ",flow)



[0 1 2 3 4 5 6]
{(6, 2), (3, 4), (4, 3), (3, 1), (6, 1), (5, 4), (0, 3), (4, 6), (6, 4), (3, 0), (4, 5), (5, 0), (2, 6), (0, 5), (1, 6), (2, 5), (1, 3), (5, 2)}
[(4, 3), (0, 3), (1, 3)]
[(3, 4), (5, 4), (6, 4)]
[(4, 5), (0, 5), (2, 5)]
[(4, 6), (2, 6), (1, 6)]
[(4, 3), (0, 3), (1, 3)]
[(3, 4), (5, 4), (6, 4)]
[(4, 5), (0, 5), (2, 5)]
[(4, 6), (2, 6), (1, 6)]
Optimal Target Flow Rate (f*): 1.9999999999932954
session:  1 edge:  (6, 2) , flow:  5.770541874314818e-13
session:  1 edge:  (3, 4) , flow:  7.441148792888982e-13
session:  1 edge:  (4, 3) , flow:  1.6688151400732444e-12
session:  1 edge:  (3, 1) , flow:  0.9999999999976044
session:  1 edge:  (6, 1) , flow:  0.9999999999956912
session:  1 edge:  (5, 4) , flow:  0.9999999999946787
session:  1 edge:  (0, 3) , flow:  0.9999999999966709
session:  1 edge:  (4, 6) , flow:  0.9999999999938102
session:  1 edge:  (6, 4) , flow:  1.1824962057463757e-12
session:  1 edge:  (3, 0) , flow:  -5.564515822620433e-14
session:  1 edge:  (4, 5) , flo